In [121]:
!pip install oracledb

In [122]:
import oracledb
import json
import random
from datetime import datetime
import hashlib 
import getpass

In [123]:
# ------------------- Conexão com o banco -------------------

In [124]:
def get_conn():
    with open("secret.txt", "r", encoding="utf-8") as f:
        creds = json.load(f)
    return oracledb.connect(
        user=creds["user"], 
        password=creds["password"], 
        dsn=creds["dsn"]
    )

In [125]:
# ------------------- Login e Cadastro -------------------

In [126]:
# Segurança de Senha

In [127]:
def hash_password(password):
    """Gera um hash SHA-256 seguro para a senha."""
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

In [128]:
def verify_password(plain_password, stored_hash):
    """Verifica se a senha digitada corresponde ao hash salvo."""
    return hash_password(plain_password) == stored_hash

In [129]:
def register_user():
    conn = get_conn()
    cur = conn.cursor()

    print("\n=== CADASTRO ===")
    nome = input("Nome completo: ").strip()

    # --- Validação de Email ---
    while True:
        email = input("Email: ").strip()
        if "@" not in email or "." not in email:
            print("❌ Email inválido. Deve conter '@' e '.'. Tente novamente.")
        else:
            break
    
    # --- Validação de CPF ---
    while True:
        cpf = input("CPF (apenas números): ").strip() 
        if not cpf.isdigit():
            print("❌ CPF inválido. Digite apenas números.")
        elif len(cpf) != 11:
            print("❌ CPF inválido. Deve conter exatamente 11 dígitos.")
        else:
            break
    
    # --- Validação de Senha ---
    while True:
        senha = getpass.getpass("Senha (mín. 6 caracteres): ").strip()
        if len(senha) < 6:
            print("❌ Senha muito curta. Deve ter no mínimo 6 caracteres.")
        else:
            break

    # --- HASH da Senha ---
    senha_hash = hash_password(senha) 

    # --- Tenta inserir no banco ---
    try:
        cur.execute("""
            INSERT INTO users (name, email, cpf, password_hash, plan_id, role)
            VALUES (:1, :2, :3, :4, 1, 'user')
        """, (nome, email, cpf, senha_hash)) 

        conn.commit()
        print("\n✅ Usuário cadastrado com sucesso (Plano Bronze).")

    except oracledb.IntegrityError as e:
        error_obj, = e.args
        if error_obj.code == 1:
             print("❌ Erro: Email ou CPF já cadastrado.")
        else:
             print(f"❌ Erro de integridade: {e}")
    
    except Exception as e:
        print(f"❌ Ocorreu um erro: {e}")

    finally:
        cur.close()
        conn.close()

In [130]:
# Célula [309c02e8] - (MODIFICAR)
def login_user():
    conn = get_conn()   
    cur = conn.cursor()

    print("\n=== LOGIN ===")
    
    while True: 
        email = input("Email (ou deixe em branco para voltar): ").strip()
        if not email:
            cur.close()
            conn.close()
            return None 

        if "@" not in email or "." not in email:
            print("❌ Formato de email inválido. Tente novamente.")
            continue 
            
        senha_digitada = getpass.getpass("Senha: ").strip()
        print("\nVerificando...")

        cur.execute("""
            SELECT 
                u.id, u.name, u.role, u.plan_id, u.selected_challenge_id,
                u.password_hash, 
                p.name AS plan_name, p.description AS plan_description,
                p.has_ai_chat, p.has_specialist_chat
            FROM users u
            JOIN plans p ON u.plan_id = p.id
            WHERE u.email = :1
        """, (email,))

        row = cur.fetchone()
        
        if row is None:
            print("❌ Email ou senha incorretos. Tente novamente.")
            print("-" * 20)
        else:
            hash_salvo = row[5] 
            
            if verify_password(senha_digitada, hash_salvo):
                break
            else:
                print("❌ Email ou senha incorretos. Tente novamente.")
                print("-" * 20)
    
    # --- Login bem-sucedido ---
    user = {
        "id": row[0],
        "name": row[1],
        "role": row[2],
        "plan_id": row[3],
        "selected_challenge": row[4],
        "plan_name": row[6],
        "plan_description": row[7],
        "ai_chat": bool(row[8]),
        "specialist_chat": bool(row[9])
    }

    cur.close()
    conn.close()

    print(f"\n✅ Login bem-sucedido! Bem-vindo, {user['name']}!")
    print(f"Seu plano: {user['plan_name']}")
    return user

In [131]:
# MENU USUÁRIO

In [132]:
def user_menu(user):
    user_id = user["id"]
    selected_challenge = user["selected_challenge"]

    while True:
        print("\n=== MENU DO USUÁRIO ===")
        print(f"Usuário: {user['name']} | Plano: {user['plan_name']}")
        print("1 - Escolher desafio")
        print("2 - Jogar desafio selecionado")
        print("3 - Ver meu progresso")
        print("4 - Mudar de Plano")  
        
        if user["ai_chat"]:
            print("5 - Chat IA (tira dúvidas)") 
        if user["specialist_chat"]:
            print("6 - Chat com Especialista") 
        print("0 - Logout")

        op = input("Escolha: ").strip()

        if op == "1":
            selected_challenge = choose_challenge(user_id)

        elif op == "2":
            if selected_challenge:
                play_selected_challenge(user_id, selected_challenge)
            else:
                print("⚠️ Nenhum desafio selecionado!")

        elif op == "3":
            view_progress(user_id)

        elif op == "4": 
            upgrade_plan(user) 

        elif op == "5" and user["ai_chat"]: 
            ia_chat_simulation()

        elif op == "6" and user["specialist_chat"]: 
            specialist_chat_simulation()

        elif op == "0":
            print("🔒 Logout efetuado.")
            break

        else:
            print("⚠️ Opção inválida!")

In [133]:
# ESCOLHER DESAFIO

In [134]:
def choose_challenge(user_id):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("SELECT id, name FROM challenges ORDER BY id")
    desafios = cur.fetchall()

    if not desafios:
        print("⚠️ Nenhum desafio disponível.")
        cur.close()
        conn.close()
        return None

    print("\n=== DESAFIOS DISPONÍVEIS ===")
    for cid, nome in desafios:
        print(f"[{cid}] {nome}")

    escolha = input("Digite o ID do desafio: ").strip()

    if not escolha.isdigit() or int(escolha) not in [d[0] for d in desafios]:
        print("⚠️ ID inválido!")
        cur.close()
        conn.close()
        return None

    cid_int = int(escolha)

    cur.execute("""
        UPDATE users SET selected_challenge_id = :1 WHERE id = :2
    """, (cid_int, user_id))

    conn.commit()
    cur.close()
    conn.close()

    print(f"✅ Desafio {cid_int} selecionado!")
    return cid_int

In [135]:
# MINI JOGOS

In [136]:
def desafio_quiz():
    perguntas = [
        ("O que é uma soft skill?", "habilidade comportamental"),
        ("O que é importante para o trabalho em equipe?", "comunicação"),
        ("O que é empatia?", "entender o outro")
    ]
    acertos = 0
    for p, resp in perguntas:
        r = input(f"{p} ").lower().strip()
        if r == resp:
            acertos += 1
    score = int((acertos / len(perguntas)) * 100)
    print(f"Pontuação: {score}%")
    return score

In [137]:
def desafio_adivinhacao():
    numero = random.randint(1, 10)
    tentativas = 0
    print("🎯 Adivinhe o número entre 1 e 10!")
    while True:
        try:
            palpite = int(input("Seu palpite: "))
        except ValueError:
            print("Digite um número válido.")
            continue

        tentativas += 1

        if palpite == numero:
            print(f"Acertou em {tentativas} tentativas!")
            return max(100 - tentativas * 10, 0)

        elif palpite < numero:
            print("🔼 Maior!")
        else:
            print("🔽 Menor!")

In [138]:
def desafio_digitar():
    frase = "Aprender é evoluir constantemente"
    print(f"Digite a seguinte frase:\n> {frase}")
    entrada = input("Digite: ").strip()
    if entrada == frase:
        print("✅ Perfeito!")
        return 100
    else:
        print("❌ Erro na digitação!")
        return 50

In [139]:
# JOGAR DESAFIO SELECIONADO

In [140]:
def play_selected_challenge(user_id, challenge_id):
    print(f"\n🎯 Jogando desafio {challenge_id}...\n")

    if challenge_id == 1:
        score = desafio_quiz()
    elif challenge_id == 2:
        score = desafio_adivinhacao()
    elif challenge_id == 3:
        score = desafio_digitar()
    else:
        print("Desafio não implementado.")
        return

    update_progress(user_id, challenge_id, score)

In [141]:
# PROGRESSO

In [142]:
def update_progress(user_id, challenge_id, score):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        MERGE INTO progress p
        USING dual
        ON (p.user_id = :1 AND p.challenge_id = :2)
        WHEN MATCHED THEN
            UPDATE SET p.score = :3, p.last_update = CURRENT_TIMESTAMP
        WHEN NOT MATCHED THEN
            INSERT (id, user_id, challenge_id, score)
            VALUES (progress_seq.nextval, :4, :5, :6)
    """, (user_id, challenge_id, score, user_id, challenge_id, score)) # <--- AQUI

    conn.commit()
    cur.close()
    conn.close()

    print("📈 Progresso atualizado!")

In [143]:
def view_progress(user_id):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT c.name, p.score, p.last_update
        FROM progress p
        JOIN challenges c ON c.id = p.challenge_id
        WHERE p.user_id = :1
        ORDER BY c.id
    """, (user_id,))

    rows = cur.fetchall()
    cur.close()
    conn.close()

    if not rows:
        print("⚠️ Nenhum progresso encontrado.")
    else:
        print("\n=== SEU PROGRESSO ===")
        for nome, score, data in rows:
            print(f"- {nome}: {score}% ({data})")

In [144]:
# MELHORAR PLANO

In [145]:
# NOVA CÉLULA
def upgrade_plan(user):
    conn = get_conn()
    cur = conn.cursor()

    current_plan_id = user["plan_id"]
    
    cur.execute("""
        SELECT id, name, price, description 
        FROM plans 
        WHERE id > :1 
        ORDER BY id
    """, (current_plan_id,))
    
    upgrades = cur.fetchall()

    if not upgrades:
        print("🎉 Você já possui o plano máximo! (Ouro)")
        cur.close()
        conn.close()
        return

    print("\n=== UPGRADE DE PLANO ===")
    print(f"Seu plano atual: {user['plan_name']}")
    print("Planos disponíveis para upgrade:")

    planos_map = {} 
    for pid, name, price, desc in upgrades:
        print(f"  [{pid}] {name} - R$ {price:.2f} ({desc})")
        planos_map[pid] = {'name': name, 'price': price}


    try:
        escolha_id = int(input("Digite o ID do plano para o qual deseja atualizar: "))
    except ValueError:
        print("⚠️ ID inválido.")
        cur.close()
        conn.close()
        return

    if escolha_id not in planos_map:
        print("⚠️ Escolha inválida.")
        cur.close()
        conn.close()
        return

    plano_escolhido = planos_map[escolha_id]
    

    print(f"\nVocê escolheu: {plano_escolhido['name']} por R$ {plano_escolhido['price']:.2f}.")
    confirm = input("Digite 'PAGAR' para confirmar o pagamento: ").strip().upper()

    if confirm != 'PAGAR':
        print("❌ Pagamento cancelado.")
        cur.close()
        conn.close()
        return


    try:
        cur.execute("UPDATE users SET plan_id = :1 WHERE id = :2", (escolha_id, user['id']))
        conn.commit()
        
        cur.execute("SELECT name, has_ai_chat, has_specialist_chat FROM plans WHERE id = :1", (escolha_id,))
        plan_row = cur.fetchone()
        
        user['plan_id'] = escolha_id
        user['plan_name'] = plan_row[0]
        user['ai_chat'] = bool(plan_row[1])
        user['specialist_chat'] = bool(plan_row[2])

        print(f"\n✅ Pagamento efetuado! Seu plano foi atualizado para {user['plan_name']}!")
        print("Seu menu será atualizado com as novas funcionalidades.")

    except Exception as e:
        conn.rollback()
        print(f"❌ Erro ao processar o pagamento: {e}")
    
    finally:
        cur.close()
        conn.close()

In [146]:
#   CHATS SIMULADOS

In [147]:
def ia_chat_simulation():
    print("\n🤖 CHAT IA — digite 'sair' para encerrar.")
    print("IA: Olá! Sou sua assistente de IA para Soft Skills.")
    print("IA: Posso te dar dicas sobre: 'comunicação', 'liderança' ou 'feedback'.")
    print("IA: Sobre o que você gostaria de perguntar?")
    
    while True:
        q = input("Você: ").strip().lower()

        if q == "sair":
            print("IA: Sessão encerrada. Continue aprendendo!")
            break
        elif "comunicação" in q:
            print("IA: Dica de Comunicação: Lembre-se de praticar a escuta ativa. ")
            print("    Isso significa focar totalmente em quem fala, em vez de \n    planejar mentalmente sua próxima resposta.")
        elif "liderança" in q:
            print("IA: Dica de Liderança: Um bom líder inspira e capacita sua equipe. ")
            print("    Delegue tarefas e confie no potencial de cada um, \n    oferecendo suporte em vez de microgerenciar.")
        elif "feedback" in q:
            print("IA: Dica de Feedback: Use a técnica do 'sanduíche': ")
            print("    Comece com um ponto positivo, apresente a crítica construtiva \n    de forma clara e termine com um reforço positivo ou plano de ação.")
        elif "ajuda" in q:
            print("IA: Tópicos que eu conheço: 'comunicação', 'liderança', 'feedback'.")
        else:
            print("IA: Entendido. Sempre tente dividir problemas grandes em partes menores para resolvê-los. ")
            print("    (Tente usar uma das palavras-chave que sugeri!)")

In [148]:
def specialist_chat_simulation():
    print("\n🎓 CHAT COM ESPECIALISTA — Conectando com um de nossos consultores de carreira...")
    print("Especialista: Olá! Que bom ver você por aqui. Estou à disposição.")
    
    while True:
        print("\nEspecialista: Sobre qual área você gostaria de um conselho prático hoje?")
        print("  [1] Como me preparar para uma entrevista de emprego?")
        print("  [2] Como lidar com conflitos na equipe?")
        print("  [3] Quero ser mais produtivo, por onde começo?")
        print("  [0] Encerrar a consulta.")
        
        op = input("Sua escolha: ").strip()

        if op == "1":
            print("\nEspecialista (Entrevista): Ótima escolha! Minhas 3 dicas de ouro são:")
            print("  1. Pesquise a empresa: entenda a cultura e os produtos dela.")
            print("  2. Prepare exemplos reais: use a 'Técnica STAR' (Situação, Tarefa, Ação, Resultado) \n     para contar suas histórias de sucesso.")
            print("  3. Faça perguntas inteligentes no final. Mostra interesse e proatividade.")
        
        elif op == "2":
            print("\nEspecialista (Conflitos): Um ponto delicado. O segredo é não evitar.")
            print("  1. Chame a pessoa para conversar em particular, nunca em público.")
            print("  2. Foque no 'problema' e nos 'fatos', nunca na 'pessoa' ou em julgamentos.")
            print("  3. Tente encontrar um objetivo em comum para resolver a questão.")

        elif op == "3":
            print("\nEspecialista (Produtividade): Comece pelo básico: 'Menos é Mais'.")
            print("  1. Use a 'Regra dos 2 Minutos': se uma tarefa leva menos de 2 min, faça agora.")
            print("  2. Defina 3 prioridades claras para o seu dia e foque nelas.")
            print("  3. Desative notificações desnecessárias. Cada interrupção custa caro ao seu foco.")

        elif op == "0":
            print("\nEspecialista: Foi um prazer! Lembre-se, o progresso é diário. Até a próxima!")
            break
        
        else:
            print("\nEspecialista: Desculpe, não entendi essa opção. Por favor, escolha um dos números do menu.")

In [149]:
# MENU ADMIN

In [150]:
def admin_menu(user):
    while True:
        print("\n=== MENU ADMIN ===")
        print("1 - Criar novo desafio")
        print("2 - Ver desafios")
        print("3 - Ver progresso de todos")
        print("4 - Gerenciar usuários")
        
        print("\n--- EXPORTAÇÕES (JSON) ---")
        print("5 - Exportar progresso de todos")
        print("6 - Exportar lista de usuários")
        print("7 - Exportar lista de desafios")
        
        print("\n0 - Sair")

        op = input("Escolha: ").strip()

        if op == "1":
            create_challenge()
        elif op == "2":
            list_challenges()
        elif op == "3":
            view_all_progress()
        elif op == "4":
            manage_users()
            
        elif op == "5":  # Antigo "4"
            export_progress_json()
        elif op == "6":  # NOVO
            export_users_json()
        elif op == "7":  # NOVO
            export_challenges_json()
            
        elif op == "0":
            break
        else:
            print("Opção inválida.")

In [151]:
def create_challenge():
    conn = get_conn()
    cur = conn.cursor()

    nome = input("Nome do desafio: ").strip()
    desc = input("Descrição: ").strip()

    cur.execute("INSERT INTO challenges (name, description) VALUES (:1, :2)", (nome, desc))

    conn.commit()
    cur.close()
    conn.close()

    print("Desafio criado!")

In [152]:
def list_challenges():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("SELECT id, name FROM challenges ORDER BY id")
    rows = cur.fetchall()

    cur.close()
    conn.close()

    print("\n=== DESAFIOS ===")
    for r in rows:
        print(f"[{r[0]}] {r[1]}")

In [153]:
def view_all_progress():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.name, c.name, p.score 
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name, c.id
    """)

    rows = cur.fetchall()
    cur.close()
    conn.close()

    print("\n=== PROGRESSO DE TODOS ===")
    for n, c, s in rows:
        print(f"{n} - {c}: {s}%")

In [154]:
def export_progress_json():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT 
            u.name AS user_name, 
            c.name AS challenge_name, 
            p.score, 
            p.last_update
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name
    """)

    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]

    cur.close()
    conn.close()

    with open("export_progress.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)

    print("JSON exportado!")


In [155]:
def export_users_json():
    """
    CONSULTA 2 (com JOIN) para exportação JSON.
    Exporta a lista de usuários e seus planos.
    """
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.id, u.name, u.email, u.role, p.name AS plan_name
        FROM users u 
        JOIN plans p ON u.plan_id = p.id
        ORDER BY u.id
    """)
    
    
    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]
    
    cur.close()
    conn.close()

    
    with open("export_users.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)

    print("✅ Arquivo 'export_users.json' gerado com sucesso!")

In [156]:
def export_challenges_json():
    """
    CONSULTA 3 (simples) para exportação JSON.
    Exporta a lista de desafios disponíveis.
    """
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("SELECT id, name, description FROM challenges ORDER BY id")
    
    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]
    
    cur.close()
    conn.close()

    with open("export_challenges.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)

    print("✅ Arquivo 'export_challenges.json' gerado com sucesso!")

In [157]:
def manage_users():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.id, u.name, u.email, u.role, p.id, p.name
        FROM users u 
        JOIN plans p ON u.plan_id = p.id
        ORDER BY u.id
    """)

    users = cur.fetchall()

    print("\n=== USUÁRIOS ===")
    for u in users:
        print(f"{u[0]} - {u[1]} | {u[2]} | role: {u[3]} | plano: {u[5]}")

    uid = input("ID do usuário (enter para sair): ").strip()
    if not uid:
        return
    if not uid.isdigit():
        print("ID inválido.")
        return

    uid_int = int(uid)

    action = input("(p)romover admin, (c)mudar plano, (d)eletar, (v)oltar: ").strip().lower()

    if action == "p":
        cur.execute("UPDATE users SET role = 'admin' WHERE id = :1", (uid_int,))
        conn.commit()
        print("Promovido a admin!")

    elif action == "c":
        cur.execute("SELECT id, name FROM plans ORDER BY id")
        plans = cur.fetchall()
        print("Planos:")
        for pid, pname in plans:
            print(f"{pid} - {pname}")
        pid = int(input("Novo plano ID: "))
        cur.execute("UPDATE users SET plan_id = :1 WHERE id = :2", (pid, uid_int))
        conn.commit()
        print("Plano alterado!")

    elif action == "d":
        conf = input("Confirmar exclusão? (s/n): ").lower()
        if conf == "s":
            cur.execute("DELETE FROM users WHERE id = :1", (uid_int,))
            conn.commit()
            print("Usuário deletado.")

    cur.close()
    conn.close()

In [158]:
# MAIN

In [159]:
def main():
    while True:
        print("\n=== SkillUp - Aprendizado Gamificado ===")
        print("1 - Registrar")
        print("2 - Login")
        print("0 - Sair")

        op = input("Escolha: ").strip()

        if op == "1":
            register_user()

        elif op == "2":
            user = login_user()
            if user:
                if user["role"] == "admin":
                    admin_menu(user)
                else:
                    user_menu(user)

        elif op == "0":
            print("Até logo!")
            break

        else:
            print("Opção inválida!")

In [160]:
if __name__ == "__main__":
    main()


=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair

=== LOGIN ===

Verificando...

✅ Login bem-sucedido! Bem-vindo, Administrador!
Seu plano: Ouro

=== MENU ADMIN ===
1 - Criar novo desafio
2 - Ver desafios
3 - Ver progresso de todos
4 - Gerenciar usuários

--- EXPORTAÇÕES (JSON) ---
5 - Exportar progresso de todos
6 - Exportar lista de usuários
7 - Exportar lista de desafios

0 - Sair

=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair
Até logo!
